# MOD_vehicle Data Exploration for LVC Toolkit Integration

This notebook explores the MOD_vehicle dataset to understand its structure and potential applications in the LVC (Live, Virtual, Constructive) Toolkit.

## Dataset Overview

The MOD_vehicle data contains multiple vehicle type classifications for acoustic and seismic detection.

### Sensor Types:
- **AUD**: Audio/Acoustic sensors (stereo)
- **EHZ**: Seismic vertical component
- **ENE**: Seismic east component  
- **ENN**: Seismic north component
- **ENZ**: Seismic Z component

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from datetime import datetime
import scipy.signal as signal
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Base path to data - server location
BASE_PATH = Path('/home/lvc_toolkit/datasets')
print(f"Base path exists: {BASE_PATH.exists()}")
print(f"Base path: {BASE_PATH}")

## 1. MOD_vehicle Dataset Structure Analysis

In [ ]:
# Explore MOD_vehicle structure
vehicle_path = BASE_PATH / 'MOD_vehicle'

print("=" * 80)
print("MOD_vehicle types:")
print("=" * 80)

vehicle_types = []
for vehicle_dir in sorted(vehicle_path.iterdir()):
    if vehicle_dir.is_dir() and not vehicle_dir.name.startswith('.'):
        # Check for subdirectories (rs1, rs2, etc.)
        subdirs = [d for d in vehicle_dir.iterdir() if d.is_dir()]
        print(f"\n{vehicle_dir.name}:")
        print(f"  Recording sessions: {len(subdirs)}")
        if subdirs:
            sample_files = list(subdirs[0].glob('*.csv'))
            print(f"  Files per session: {[f.name for f in sample_files[:5]]}")  # Show first 5
        vehicle_types.append(vehicle_dir.name)

print(f"\nTotal vehicle types: {len(vehicle_types)}")
print(f"Vehicle types: {vehicle_types}")

## 2. Data Format Analysis

Let's examine the format of the seismic and audio data files.

In [ ]:
# Load and analyze a seismic file from vehicle data
sample_vehicle = vehicle_path / 'mustang' / 'rs1' / 'ehz.csv'

print("Seismic Data Format Analysis (MOD_vehicle):")
print("=" * 80)

# Note: Seismic CSV format is [amplitude timestamp] with trailing space
seismic_df = pd.read_csv(sample_vehicle, sep='\s+', header=None, names=['value', 'timestamp'])
print(f"Shape: {seismic_df.shape}")
print(f"\nFirst few rows:")
print(seismic_df.head(10))
print(f"\nData statistics:")
print(seismic_df.describe())
print(f"\nSample rate: ~{1/seismic_df['timestamp'].diff().mean():.2f} Hz")

## 3. Data Cleaning and Preprocessing

Before analyzing the signals, we need to address data quality issues:
- **DC Offset Removal**: Remove mean value to center signals around zero
- **Outlier Detection**: Identify and handle anomalous values
- **Data Quality Checks**: Verify signal integrity

In [ ]:
def remove_dc_offset(signal_data):
    """Remove DC offset by subtracting the mean"""
    return signal_data - np.mean(signal_data)

def detect_outliers(signal_data, threshold=5):
    """Detect outliers using z-score method"""
    mean = np.mean(signal_data)
    std = np.std(signal_data)
    z_scores = np.abs((signal_data - mean) / std)
    outlier_indices = np.where(z_scores > threshold)[0]
    return outlier_indices

def remove_outliers(signal_data, threshold=5, method='clip'):
    """Remove or clip outliers
    
    Parameters:
    - method: 'clip' (replace with threshold) or 'interpolate' (linear interpolation)
    """
    outlier_indices = detect_outliers(signal_data, threshold)
    cleaned = signal_data.copy()
    
    if method == 'clip':
        # Clip to +/- threshold standard deviations
        mean = np.mean(signal_data)
        std = np.std(signal_data)
        cleaned = np.clip(cleaned, mean - threshold*std, mean + threshold*std)
    elif method == 'interpolate' and len(outlier_indices) > 0:
        # Linear interpolation for outliers
        mask = np.ones(len(signal_data), dtype=bool)
        mask[outlier_indices] = False
        good_indices = np.where(mask)[0]
        cleaned[outlier_indices] = np.interp(outlier_indices, good_indices, signal_data[good_indices])
    
    return cleaned, outlier_indices

def check_data_quality(signal_data, signal_name="Signal"):
    """Perform comprehensive data quality checks"""
    print(f"\n{signal_name} Quality Check:")
    print("=" * 60)
    
    # Basic statistics
    print(f"  Samples: {len(signal_data)}")
    print(f"  Mean (DC offset): {np.mean(signal_data):.4f}")
    print(f"  Std Dev: {np.std(signal_data):.4f}")
    print(f"  Min: {np.min(signal_data):.4f}")
    print(f"  Max: {np.max(signal_data):.4f}")
    print(f"  Range: {np.ptp(signal_data):.4f}")
    
    # Check for NaN/Inf
    nan_count = np.sum(np.isnan(signal_data))
    inf_count = np.sum(np.isinf(signal_data))
    print(f"  NaN values: {nan_count}")
    print(f"  Inf values: {inf_count}")
    
    # Outlier detection
    outliers = detect_outliers(signal_data, threshold=5)
    print(f"  Outliers (>5σ): {len(outliers)} ({100*len(outliers)/len(signal_data):.2f}%)")
    
    # Signal-to-noise ratio estimate (using RMS)
    rms = np.sqrt(np.mean(signal_data**2))
    noise_estimate = np.std(signal_data[:100])  # Assuming first 100 samples might be noise
    if noise_estimate > 0:
        snr = 20 * np.log10(rms / noise_estimate)
        print(f"  Estimated SNR: {snr:.2f} dB")
    
    return {
        'mean': np.mean(signal_data),
        'std': np.std(signal_data),
        'outliers': len(outliers),
        'nan_count': nan_count,
        'inf_count': inf_count
    }

print("Data cleaning functions loaded successfully!")

In [ ]:
# Test data cleaning on seismic data from MOD_vehicle
print("RAW DATA ANALYSIS:")
raw_quality = check_data_quality(seismic_df['value'].values, "Raw Seismic EHZ")

# Apply DC offset removal
seismic_cleaned = remove_dc_offset(seismic_df['value'].values)
print("\n\nAFTER DC OFFSET REMOVAL:")
dc_quality = check_data_quality(seismic_cleaned, "DC-Corrected Seismic EHZ")

# Remove outliers
seismic_cleaned_no_outliers, outlier_idx = remove_outliers(seismic_cleaned, threshold=5, method='clip')
print("\n\nAFTER OUTLIER REMOVAL:")
final_quality = check_data_quality(seismic_cleaned_no_outliers, "Fully Cleaned Seismic EHZ")

print(f"\n\nCLEANING SUMMARY:")
print(f"  DC offset removed: {raw_quality['mean']:.4f}")
print(f"  Outliers detected: {len(outlier_idx)}")
print(f"  Data quality improved: ✓")

In [ ]:
# Visualize cleaning effects on MOD_vehicle data
fig, axes = plt.subplots(3, 2, figsize=(16, 12))

time_rel = seismic_df['timestamp'] - seismic_df['timestamp'].iloc[0]

# Raw data - Time series
axes[0, 0].plot(time_rel, seismic_df['value'], linewidth=0.8, color='red', alpha=0.7)
axes[0, 0].set_title('Raw Data - Time Series', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Amplitude')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=np.mean(seismic_df['value']), color='black', linestyle='--', label=f'Mean: {np.mean(seismic_df["value"]):.2f}')
axes[0, 0].legend()

# Raw data - Histogram
axes[0, 1].hist(seismic_df['value'], bins=100, color='red', alpha=0.6, edgecolor='black')
axes[0, 1].set_title('Raw Data - Distribution', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Amplitude')
axes[0, 1].axvline(x=np.mean(seismic_df['value']), color='black', linestyle='--', linewidth=2)
axes[0, 1].grid(True, alpha=0.3)

# DC corrected - Time series
axes[1, 0].plot(time_rel, seismic_cleaned, linewidth=0.8, color='orange', alpha=0.7)
axes[1, 0].set_title('After DC Offset Removal - Time Series', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Amplitude')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='black', linestyle='--', label='Mean: 0.00')
axes[1, 0].legend()

# DC corrected - Histogram
axes[1, 1].hist(seismic_cleaned, bins=100, color='orange', alpha=0.6, edgecolor='black')
axes[1, 1].set_title('After DC Offset Removal - Distribution', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Amplitude')
axes[1, 1].axvline(x=0, color='black', linestyle='--', linewidth=2)
axes[1, 1].grid(True, alpha=0.3)

# Fully cleaned - Time series
axes[2, 0].plot(time_rel, seismic_cleaned_no_outliers, linewidth=0.8, color='green', alpha=0.7)
axes[2, 0].set_title('After Outlier Removal - Time Series', fontsize=12, fontweight='bold')
axes[2, 0].set_xlabel('Time (seconds)')
axes[2, 0].set_ylabel('Amplitude')
axes[2, 0].grid(True, alpha=0.3)
axes[2, 0].axhline(y=0, color='black', linestyle='--')

# Fully cleaned - Histogram
axes[2, 1].hist(seismic_cleaned_no_outliers, bins=100, color='green', alpha=0.6, edgecolor='black')
axes[2, 1].set_title('After Outlier Removal - Distribution', fontsize=12, fontweight='bold')
axes[2, 1].set_xlabel('Amplitude')
axes[2, 1].axvline(x=0, color='black', linestyle='--', linewidth=2)
axes[2, 1].grid(True, alpha=0.3)

fig.suptitle('Data Cleaning Pipeline - Seismic EHZ (Mustang)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## 4. Signal Visualization

Analyze cleaned seismic data from vehicle recordings.

In [ ]:
# Visualize cleaned seismic data
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

time_rel = seismic_df['timestamp'] - seismic_df['timestamp'].iloc[0]

# Time series
axes[0].plot(time_rel, seismic_cleaned_no_outliers, linewidth=0.8, color='green')
axes[0].set_title('Seismic EHZ - Mustang (Cleaned)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Time (seconds)')
axes[0].set_ylabel('Amplitude')
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.3, linewidth=1)
axes[0].grid(True, alpha=0.3)

# Histogram
axes[1].hist(seismic_cleaned_no_outliers, bins=100, color='green', alpha=0.7, edgecolor='black')
axes[1].set_title('Amplitude Distribution (Cleaned)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Amplitude')
axes[1].set_ylabel('Frequency')
axes[1].axvline(x=0, color='black', linestyle='--', alpha=0.5, linewidth=2)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Frequency Domain Analysis

In [ ]:
# Compute FFT for seismic data
def compute_spectrum(signal_data, sample_rate):
    """Compute frequency spectrum using FFT"""
    N = len(signal_data)
    yf = fft(signal_data)
    xf = fftfreq(N, 1/sample_rate)[:N//2]
    power = 2.0/N * np.abs(yf[:N//2])
    return xf, power

# Compute spectrum for cleaned seismic data
sample_rate = 1 / seismic_df['timestamp'].diff().mean()
freq, power = compute_spectrum(seismic_cleaned_no_outliers, sample_rate)

fig, ax = plt.subplots(1, 1, figsize=(14, 6))

ax.semilogy(freq, power, linewidth=0.8, color='green', alpha=0.7)
ax.set_title('Seismic Frequency Spectrum - Mustang (Cleaned)', fontsize=14, fontweight='bold')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power (log scale)')
ax.set_xlim([0, 50])  # Focus on typical seismic frequencies
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create spectrogram for seismic data
fig, ax = plt.subplots(1, 1, figsize=(14, 6))

f, t, Sxx = signal.spectrogram(seismic_cleaned_no_outliers, sample_rate, nperseg=512)
im = ax.pcolormesh(t, f, 10 * np.log10(Sxx + 1e-10), shading='gouraud', cmap='viridis')
ax.set_ylabel('Frequency (Hz)')
ax.set_xlabel('Time (seconds)')
ax.set_title('Spectrogram - Seismic EHZ (Mustang, Cleaned)', fontsize=14, fontweight='bold')
ax.set_ylim([0, 50])
plt.colorbar(im, ax=ax, label='Power (dB)')

plt.tight_layout()
plt.show()

## 6. Multi-Sensor Comparison

In [ ]:
# Load all seismic sensors for comparison (with cleaning)
sensor_types = ['ehz', 'ene', 'enn', 'enz']
seismic_data = {}
seismic_data_cleaned = {}

for sensor in sensor_types:
    file_path = vehicle_path / 'mustang' / 'rs1' / f'{sensor}.csv'
    if file_path.exists():
        df = pd.read_csv(file_path, sep='\s+', header=None, names=['value', 'timestamp'])
        seismic_data[sensor] = df
        
        # Clean the data
        cleaned = remove_dc_offset(df['value'].values)
        cleaned, _ = remove_outliers(cleaned, threshold=5, method='clip')
        seismic_data_cleaned[sensor] = cleaned
        
        print(f"Loaded and cleaned {sensor.upper()}: {len(df)} samples")

print(f"\nLoaded {len(seismic_data)} seismic sensors")

In [ ]:
# Visualize all seismic sensors (cleaned)
fig, axes = plt.subplots(len(seismic_data), 1, figsize=(14, 12))

colors = ['green', 'orange', 'purple', 'brown']

for (sensor, df), cleaned, ax, color in zip(seismic_data.items(), seismic_data_cleaned.values(), axes, colors):
    time_rel = df['timestamp'] - df['timestamp'].iloc[0]
    ax.plot(time_rel, cleaned, linewidth=0.8, color=color, label=sensor.upper())
    ax.set_title(f'Seismic Sensor: {sensor.upper()} (Cleaned)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Amplitude')
    ax.axhline(y=0, color='black', linestyle='--', alpha=0.3, linewidth=1)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')

axes[-1].set_xlabel('Time (seconds)')
fig.suptitle('Multi-Sensor Seismic Data Comparison - Mustang (Cleaned)', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

## 7. Vehicle Classification Analysis

**Note:** All features extracted from cleaned data (DC offset removed, outliers clipped).

In [ ]:
# Extract features from different vehicle types (with cleaning)
vehicle_features = []

sample_vehicles = ['bicycle', 'motor', 'mustang', 'pickup', 'scooter', 'tesla', 'walk']

for vehicle in sample_vehicles:
    vehicle_dir = vehicle_path / vehicle
    if vehicle_dir.exists():
        # Get first recording session
        sessions = [d for d in vehicle_dir.iterdir() if d.is_dir()]
        if sessions:
            ehz_file = sessions[0] / 'ehz.csv'
            if ehz_file.exists():
                # Read first 100000 samples for consistency
                df = pd.read_csv(ehz_file, sep='\s+', header=None, names=['value', 'timestamp'], nrows=100000)
                
                # Clean the data
                cleaned_signal = remove_dc_offset(df['value'].values)
                cleaned_signal, _ = remove_outliers(cleaned_signal, threshold=5, method='clip')
                
                # Calculate features on cleaned data
                energy = np.sum(cleaned_signal ** 2)
                rms = np.sqrt(np.mean(cleaned_signal ** 2))
                peak = np.max(np.abs(cleaned_signal))
                std = np.std(cleaned_signal)
                
                # Frequency domain features
                sr = 1 / df['timestamp'].diff().mean()
                freqs, power = compute_spectrum(cleaned_signal, sr)
                dominant_freq = freqs[np.argmax(power)]
                
                vehicle_features.append({
                    'vehicle': vehicle,
                    'energy': energy,
                    'rms': rms,
                    'peak': peak,
                    'std': std,
                    'dominant_freq': dominant_freq
                })
                print(f"Processed (cleaned): {vehicle}")

vehicle_df = pd.DataFrame(vehicle_features)
print("\nVehicle Type Features (from cleaned data):")
print(vehicle_df.round(2))

In [ ]:
# Visualize vehicle type characteristics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# RMS comparison
axes[0, 0].bar(vehicle_df['vehicle'], vehicle_df['rms'], color='skyblue', edgecolor='black')
axes[0, 0].set_title('RMS Amplitude by Vehicle Type', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('RMS')
axes[0, 0].tick_params(axis='x', rotation=45)
axes[0, 0].grid(axis='y', alpha=0.3)

# Peak amplitude comparison
axes[0, 1].bar(vehicle_df['vehicle'], vehicle_df['peak'], color='coral', edgecolor='black')
axes[0, 1].set_title('Peak Amplitude by Vehicle Type', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Peak')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(axis='y', alpha=0.3)

# Energy comparison (log scale)
axes[1, 0].bar(vehicle_df['vehicle'], vehicle_df['energy'], color='lightgreen', edgecolor='black')
axes[1, 0].set_yscale('log')
axes[1, 0].set_title('Signal Energy by Vehicle Type (log scale)', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Energy')
axes[1, 0].tick_params(axis='x', rotation=45)
axes[1, 0].grid(axis='y', alpha=0.3)

# Dominant frequency comparison
axes[1, 1].bar(vehicle_df['vehicle'], vehicle_df['dominant_freq'], color='plum', edgecolor='black')
axes[1, 1].set_title('Dominant Frequency by Vehicle Type', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Frequency (Hz)')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 8. LVC Toolkit Integration Recommendations

### Key Findings:

1. **Multi-Modal Sensor Data**
   - Audio (AUD): Stereo acoustic data at 16 kHz (some vehicles)
   - Seismic (EHZ, ENE, ENN, ENZ): Ground vibration sensors at ~100 Hz
   - Complementary information for robust detection

2. **Vehicle Classification**
   - Different vehicles produce distinct acoustic/seismic signatures
   - Features: RMS, peak amplitude, energy, spectral content
   - Machine learning potential for automatic classification

3. **Dataset Composition**
   - Multiple vehicle types: bicycle, motor, mustang, pickup, scooter, tesla, walk, and more
   - Multiple recording sessions per vehicle type
   - Consistent sensor array across recordings

### LVC Toolkit Applications:

#### 1. **Live Component**
   - Real-time vehicle detection and tracking
   - Ground truth data for validation
   - Sensor fusion algorithms

#### 2. **Virtual Component**
   - Signal simulation based on real data
   - Physics-based propagation models
   - Synthetic data generation for training

#### 3. **Constructive Component**
   - Agent behavior modeling
   - Detection probability models
   - Mission planning and analysis

### Recommended Features for ML Models:

1. **Time Domain:**
   - RMS amplitude
   - Peak values
   - Zero-crossing rate
   - Signal energy
   
2. **Frequency Domain:**
   - Dominant frequency
   - Spectral centroid
   - Spectral rolloff
   - MFCCs (for audio)
   
3. **Multi-Sensor:**
   - Cross-correlation between sensors
   - Sensor fusion features
   - Directional information (from E/N components)

## 9. Feature Extraction for ML Pipeline

In [ ]:
def extract_features(signal_data, sample_rate=100):
    """Extract comprehensive features from signal data"""
    features = {}
    
    # Time domain features
    features['mean'] = np.mean(signal_data)
    features['std'] = np.std(signal_data)
    features['rms'] = np.sqrt(np.mean(signal_data ** 2))
    features['peak'] = np.max(np.abs(signal_data))
    features['peak_to_peak'] = np.ptp(signal_data)
    features['energy'] = np.sum(signal_data ** 2)
    features['zero_crossing_rate'] = np.sum(np.diff(np.sign(signal_data)) != 0) / len(signal_data)
    
    # Statistical features
    features['skewness'] = pd.Series(signal_data).skew()
    features['kurtosis'] = pd.Series(signal_data).kurtosis()
    features['q25'] = np.percentile(signal_data, 25)
    features['q75'] = np.percentile(signal_data, 75)
    
    # Frequency domain features
    freqs, power = compute_spectrum(signal_data, sample_rate)
    features['dominant_freq'] = freqs[np.argmax(power)]
    features['spectral_centroid'] = np.sum(freqs * power) / np.sum(power)
    cumsum = np.cumsum(power)
    features['spectral_rolloff'] = freqs[np.where(cumsum >= 0.85 * cumsum[-1])[0][0]]
    features['spectral_bandwidth'] = np.sqrt(np.sum(((freqs - features['spectral_centroid']) ** 2) * power) / np.sum(power))
    
    return features

# Test feature extraction on cleaned data
test_signal = seismic_cleaned_no_outliers[:10000]
test_features = extract_features(test_signal)

print("Extracted Features (from cleaned data):")
print("=" * 60)
for key, value in test_features.items():
    print(f"{key:25s}: {value:12.4f}")

## 10. Next Steps for LVC Integration

### Immediate Actions:
1. Build feature extraction pipeline for all vehicle datasets
2. Create labeled dataset for vehicle classification
3. Train baseline ML models (Random Forest, SVM, Neural Networks)
4. Develop real-time detection algorithms
5. Integrate with M3NVC dataset for expanded training data

### Integration Tasks:
1. **Data Interface**: Create standardized data loaders for LVC Toolkit
2. **Feature Service**: Deploy feature extraction as microservice
3. **Classification Service**: Real-time vehicle classification API
4. **Visualization Dashboard**: Live monitoring and analysis
5. **Simulation Module**: Generate synthetic FOCAL-like data

### Performance Metrics:
- Classification accuracy by vehicle type
- Processing latency for real-time operation
- False alarm rate
- Sensor fusion performance gain
- Cross-dataset generalization (MOD_vehicle + M3NVC)

In [ ]:
# Summary statistics
print("MOD_vehicle Data Exploration Summary")
print("=" * 80)
print(f"\nDataset Structure:")
print(f"  Vehicle Types: {len(vehicle_types)}")
print(f"  Sensor Modalities: Audio + Seismic (EHZ, ENE, ENN, ENZ)")
print(f"\nData Characteristics:")
print(f"  Seismic Sample Rate: ~100 Hz")
print(f"  Audio Sample Rate: 16,000 Hz (where available)")
print(f"  Seismic Channels: 4 (3D + vertical)")
print(f"\nVehicle Types Analyzed: {', '.join(vehicle_df['vehicle'].values)}")
print(f"\nServer Path: {BASE_PATH}")
print(f"\nReady for LVC Toolkit integration!")